<a href="https://colab.research.google.com/github/teja1091/road-lane-detection/blob/main/roadlane_det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless numpy


In [ ]:
import cv2
import numpy as np

def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to the image
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # Detect edges using Canny edge detector
    edges = cv2.Canny(blur, 50, 150)
    return edges

def region_of_interest(edges):
    # Define a mask to focus on the region of interest (ROI)
    height, width = edges.shape
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (0, height),
        (width, height),
        (width, int(height * 0.6)),
        (0, int(height * 0.6)),
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    masked_edges = cv2.bitwise_and(edges, mask)
    return masked_edges

def detect_lines(edges):
    # Use Hough Transform to detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, maxLineGap=50)
    return lines

def display_lines(image, lines):
    # Create an image to draw the lines on
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 10)
    # Combine the original image with the line image
    combined_image = cv2.addWeighted(image, 0.8, line_image, 1, 1)
    return combined_image

def lane_detection(image_path, output_path):
    # Read the input image
    image = cv2.imread(image_path)

    # Check if the image was successfully loaded
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return

    # Preprocess the image to detect edges
    edges = preprocess_image(image)
    # Focus on the region of interest
    roi_edges = region_of_interest(edges)
    # Detect lines in the image
    lines = detect_lines(roi_edges)
    # Display lines on the original image
    output_image = display_lines(image, lines)
    # Save the output image
    cv2.imwrite(output_path, output_image)
    print(f"Output image saved at {output_path}")

# Example usage
input_image_path = '/content/sample_data/input.jpeg'
output_image_path = 'output.jpg'
lane_detection(input_image_path, output_image_path)
